# Low-level hackery

_(nothing to install; just numpy, matplotlib from before)_

One function I've used without much comment is `numpy.frombuffer`, which lets us wrap arbitrary regions of memory as Numpy arrays. We can "peek" at any memory we want; we can also "poke" it, changing values, byte by byte.

Consider, for instance, a byte string. These are immutable (cannot be changed) in Python:

In [ ]:
hello = b"Hello, world!"

In [ ]:
try:
    hello[4:8] = b"????"
except TypeError as err:
    print("Nope: " + str(err))

In [ ]:
import numpy
a = numpy.frombuffer(hello, dtype=numpy.uint8)
a

In [ ]:
a.view("S1")

By default, Numpy tries to protect you from doing evil things.

In [ ]:
try:
    a[4:8] = [69, 86, 73, 76]
except ValueError as err:
    print("Nope: " + str(err))

But this is Python: we can shoot our feet if we want to.

In [ ]:
a.flags.writeable = True

In [ ]:
a[4:8] = [69, 86, 73, 76]

In [ ]:
hello

This messes with Python's internal data model.

In [ ]:
hello = b"Hello, world!"
a = numpy.frombuffer(hello, dtype=numpy.uint8)
a.flags.writeable = True
a[4:8] = [69, 86, 73, 76]
print(hello == b"Hello, world!")

In [ ]:
exec("""
hello = b"Hello, world!"
a = numpy.frombuffer(hello, dtype=numpy.uint8)
a.flags.writeable = True
a[4:8] = [69, 86, 73, 76]
print(hello == b"Hello, world!")
""")

(The second example was interpreted as a `.pyc` script, in which all instances of the literal `b"Hello, world!"` were replaced by a single object: modifying that object in line 4 changed it in line 5!)

With the help of ctypes, a built-in Python library, Numpy can wrap any address at all. (Some will cause segmentation faults, so be careful!)

In [ ]:
x = 12345

In [ ]:
import ctypes
import sys

ptr = ctypes.cast(id(x), ctypes.POINTER(ctypes.c_uint8))
a = numpy.ctypeslib.as_array(ptr, (sys.getsizeof(x),))
a

Do you see it? We're looking at a Python object header, a pointer to the `int` type (also a Python object), and then the number itself. Here's a hint: it's the last four bytes.

In [ ]:
a[-4:].view(numpy.int32)

Let's try a string.

In [ ]:
y = "Hey there."
ptr = ctypes.cast(id(y), ctypes.POINTER(ctypes.c_uint8))
a = numpy.ctypeslib.as_array(ptr, (sys.getsizeof(y),))
a

In [ ]:
a[-11:].tostring()

By wrapping a pointer as a Numpy array (and maybe setting `array.flags.writeable = True`), we can do anything.

We can break anything.

Seriously, just letting `numpy.ctypeslib.as_array(0, (8,))` repr itself on the screen would cause a segmentation fault. 

Is there anything useful we can do with this power?

The original purpose of the ctypes library was to run code in compiled C libraries (not C++, that's much more complicated).

For a busy data analyst, the advantage of that is that you don't need to write (or wait for) official bindings to use a C library.

In [ ]:
import ctypes
libc = ctypes.cdll.LoadLibrary("/lib/x86_64-linux-gnu/libc.so.6")

We can run arbitrary functions from `libc.so.6`, but we have to tell Python what its argument types are (that's not stored in the shared object file).

In [ ]:
libc.malloc.argtypes = (ctypes.c_size_t,)                      # argument types (only one)
libc.malloc.restype = ctypes.POINTER(ctypes.c_double)          # return type

In [ ]:
ptr = libc.malloc(100 * numpy.dtype(numpy.float64).itemsize)   # pass number of bytes
ptr

In [ ]:
a = numpy.ctypeslib.as_array(ptr, (100,))
a

In [ ]:
a[:] = 0.0
a

In [ ]:
a.flags

We've just used low-level `libc.malloc` to allocate data for an array. This array doesn't "own" its memory, so it doesn't go away when the Python object (`a`) gets garbage collected. We'd have to call `libc.free`, like any C programmer.

Why do this? I've used it to allocate arrays on NUMA hardware and Knight's Landing MCDRAM. Specialty memory allocations will probably get more important, not less, as architectures get more heterogeneous.

(No reason to write an entire analysis in C just to get special allocators.)

Another possible reason is to access special devices; device drivers are often written in C and distributed with C programmers in mind, but they may be the only thing between you and an important dataset.

Another example: you can wrap structs. (The snake eats its own tail again.)

In [ ]:
class PyObject(ctypes.Structure): pass
PyObject._fields_ = [("ob_refcnt", ctypes.c_size_t),
                     ("ob_type", ctypes.POINTER(PyObject))]

which is equivalent to

```c
struct PyObject {
    size_t ob_refcnt;
    PyObject* ob_type;
    // there's more, but you only need to encode as much as you need...
}
```

In [ ]:
hello = b"Hello, world!"

In [ ]:
ptr = PyObject.from_address(id(hello))
ptr

In [ ]:
ptr.ob_refcnt

This `ob_refcnt` is the number of Python references to a given object. There's a way to do it with a `sys` call:

In [ ]:
sys.getrefcount(hello)

but it's always one too high because you create a reference to pass it to that function!

In [ ]:
biglist = [hello] * 1000

In [ ]:
ptr.ob_refcnt

In [ ]:
sys.getrefcount(hello)

The ctypes library can wrap numbers, strings, pointers, arrays, and structs, which is just about everything you might encounter in C. The reason C++ isn't supported is because C shoehorns its much larger type system into shared object files by "name mangling."

Even if you reverse the name mangling with the `c++filt` program, those names cannot be uniquely identified without interpreting the C++ headers. By that point, you might as well use Cython or pybind11.

**One last example:** the quickest, dirtiest way possible to call out to compiled code:

In [ ]:
import os
with open("tmp.cpp", "w") as cfile:
    cfile.write("""
#include <complex>
extern "C" {    
    void quick(int height, int width, int maxiterations, double* c, int* fractal) {
        for (int h = 0;  h < height;  h++) {
            for (int w = 0;  w < width;  w++) {
                double creal = c[2 * (h + height*w)];
                double cimag = c[2 * (h + height*w) + 1];
                std::complex<double> ci = std::complex<double>(creal, cimag);
                std::complex<double> z = ci;
                for (int i = 0;  i < maxiterations;  i++) {
                    z = z * z + ci;
                    if (std::abs(z) > 2) {
                        fractal[h + height*w] = i;
                        break;
                    }
                }
            }
        }
    }
}
""")
assert os.system("gcc -O3 -fPIC -shared tmp.cpp -o libtmp.so") == 0

In [ ]:
libtmp = ctypes.cdll.LoadLibrary(os.path.join(os.getcwd(), "libtmp.so"))
libtmp.quick.argtypes = (ctypes.c_int, ctypes.c_int, ctypes.c_int,
                         ctypes.POINTER(ctypes.c_double),
                         ctypes.POINTER(ctypes.c_int))
libtmp.quick.restype = None

In [ ]:
def run_dirty(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    libtmp.quick(height, width, maxiterations,
                 ctypes.cast(c.ctypes.data, ctypes.POINTER(ctypes.c_double)),
                 ctypes.cast(fractal.ctypes.data, ctypes.POINTER(ctypes.c_int)))
    return fractal

In [ ]:
import time
starttime = time.time()
fractal = run_dirty(8000, 12000)
time.time() - starttime

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(fractal)
# ax.imshow(fractal[-2000:, :3000])

Not very good time, but the right answer.

With that monstrosity, I'll end the workshop.

Cheers!